# Dense Retrieval

### Setup
Load needed API keys and relevant Python libaries.

In [1]:
%pip install cohere
%pip install weaviate-client Annoy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(dotenv_path=find_dotenv())  # read local .env file

In [3]:
import cohere
COHERE_API_KEY = "Replace with your COHERE API key"

co = cohere.Client(api_key=COHERE_API_KEY)

In [4]:
import weaviate

WEAVIATE_API_KEY = "76320a90-53d8-42bc-b41d-678647c6672e"
 
auth_config = weaviate.auth.AuthApiKey(
    api_key=WEAVIATE_API_KEY)  # os.environ['WEAVIATE_API_KEY']

In [5]:
COHERE_API_KEY = "d2DdEUwMTAMkGAMTR5SKyZ8hoG8dgYD5oY7LBNm2"
url = "https://cohere-demo.weaviate.network/"

client = weaviate.Client(
    url=url, # os.environ["WEAVIATE_API_URL"]
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": COHERE_API_KEY, # os.environ['COHERE_API_KEY']
    }
)

client.is_ready()  # check if True

/home/kaushik/miniconda3/envs/py3/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


True

## Part 1: Vector Database for Semantic Search

In [6]:
def dense_retrieval(query,
                    results_lang="en",
                    properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
                    num_results=5):

    nearText = {"concepts": [query]}

    # To filter by languague
    where_filter = {
        "path": ["lang"],
        "operator": "Equal",
        "valueString": results_lang
    }

    response = (
        client.query
        .get("Articles", properties)
        .with_near_text(nearText)
        .with_where(where_filter)
        .with_limit(num_results)
        .do()
    )

    result = response["data"]["Get"]["Articles"]

    return result

In [7]:
from utils import print_result

/home/kaushik/miniconda3/envs/py3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kaushik/miniconda3/envs/py3/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


### Basic Query

In [8]:
query = "Who wrote Hamlet?"
dense_retrieval_results = dense_retrieval(query=query)
print_result(result=dense_retrieval_results)

item: 0
_additional: {'distance': -154.75615}

lang: en

text: There are many works that have been pointed to as possible sources for Shakespeare's play—from ancient Greek tragedies to Elizabethan plays. The editors of the Arden Shakespeare question the idea of "source hunting", pointing out that it presupposes that authors always require ideas from other works for their own, and suggests that no author can have an original idea or be an originator. When Shakespeare wrote there were many stories about sons avenging the murder of their fathers, and many about clever avenging sons pretending to be foolish in order to outsmart their foes. This would include the story of the ancient Roman, Lucius Junius Brutus, which Shakespeare apparently knew, as well as the story of Amleth, which was preserved in Latin by 13th-century chronicler Saxo Grammaticus in his "Gesta Danorum", and printed in Paris in 1514. The Amleth story was subsequently adapted and then published in French in 1570 by the 16t

### Medium Query

In [9]:
query = "What is the captial of Canada?"
dense_retrieval_results = dense_retrieval(query=query)
print_result(result=dense_retrieval_results)

item: 0
_additional: {'distance': -154.76201}

lang: en

text: A number of buildings across Canada are reserved by the Crown for the use of the monarch and his viceroys. Each is called "Government House", but may be customarily known by some specific name. The sovereign's and governor general's official residences are Rideau Hall in Ottawa and the Citadelle in Quebec City. Each of these royal seats holds pieces from the Crown Collection. Further, though neither was ever used for their intended purpose, Hatley Castle in British Columbia was purchased in 1940 by King George VI in Right of Canada to use as his home during the course of the Second World War and the Emergency Government Headquarters, built in 1959 at CFS Carp and decommissioned in 1994, included a residential apartment for the sovereign or governor general in the case of a nuclear attack on Ottawa.

title: Monarchy of Canada

url: https://en.wikipedia.org/wiki?curid=56504

views: 2000


item: 1
_additional: {'distance': -15

### Complicated Query

In [10]:
from utils import keyword_search

query = "Tallest person in history?"
keyword_search_results = keyword_search(query=query, client=client)
print_result(result=keyword_search_results)

item: 0
text: The population of Japan peaked at 128,083,960 in 2008. It had decreased by 2,373,960 by December 2020. In 2011, the economy of China became the world's second largest. Japan's economy descended to third largest by nominal GDP. Despite Japan's economic difficulties, this period also saw Japanese popular culture, including video games, anime, and manga, expanding worldwide, especially among young people. In March 2011, the Tokyo Skytree became the tallest tower in the world at , displacing the Canton Tower. It is the second tallest structure in the world after the Burj Khalifa ().

title: History of Japan

url: https://en.wikipedia.org/wiki?curid=25890428


item: 1
text: Alpinism author Jon Krakauer (1997) wrote in "Into Thin Air" that it would be a bigger challenge to climb the second-highest peak of each continent, known as the Seven Second Summits – a feat that was not accomplished until January 2013. This discussion had previously been published in an article titled "Th

In [11]:
query = "Tallest person in history?"
dense_retrieval_results = dense_retrieval(query=query)
print_result(result=dense_retrieval_results)

item: 0
_additional: {'distance': -147.6825}

lang: en

text: Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri.

title: Robert Wadlow

url: https://en.wikipedia.org/wiki?curid=359117

views: 3000


item: 1
_additional: {'distance': -147.38083}

lang: en

text: Bol came from a family of extraordinarily tall men and women. He said: "My mother was , my father , and my sister is . And my great-grandfather was even taller—." His ethnic group, the Dinka, and the Nilotic people of which they are a part, are among the tallest populations in the world. Bol's hometown, Turalei, is the origin of other exceptionally tall people, including basketball player Ring Ayuel. "I was born in a village, where you cannot measure yourself," Bol reflected. "I learned I w

Query in Arabic

In [12]:
query = "أطول رجل في التاريخ"
dense_retrieval_results = dense_retrieval(query=query)
print_result(result=dense_retrieval_results)

item: 0
_additional: {'distance': -147.44199}

lang: en

text: Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri.

title: Robert Wadlow

url: https://en.wikipedia.org/wiki?curid=359117

views: 3000


item: 1
_additional: {'distance': -147.09518}

lang: en

text: Kösen turned 40 years old on 10 December 2022. He celebrated his birthday a few days early by visiting the Ripley's Believe It or Not! museum in Orlando, Florida, USA and posing next to a life-sized statue of Robert Wadlow, the tallest man ever at 272 cm (8 ft 11.1 in).

title: Sultan Kösen

url: https://en.wikipedia.org/wiki?curid=8445237

views: 2000


item: 2
_additional: {'distance': -146.9144}

lang: en

text: Bol and Gheorghe Mureșan are the two tallest players in the history of the 

In [13]:
query = "film about a time travel paradox"
dense_retrieval_results = dense_retrieval(query=query)
print_result(result=dense_retrieval_results)

item: 0
_additional: {'distance': -151.41705}

lang: en

text: Although the production contracted out various effect houses to try to make the time travelling effects feel like more of a spectacle, they found the resulting work "just completely wrong" tonally and instead focused on a more low-key approach. Curtis has opined "that in the end it turns out to be a kind of anti–time travel time travel movie. It uses all the time travel stuff but without it feeling like it's a science fiction thing particularly or without it feeling that time travel can actually solve your life."

title: About Time (2013 film)

url: https://en.wikipedia.org/wiki?curid=38094431

views: 2000


item: 1
_additional: {'distance': -150.89001}

lang: en

text: The film was directed by Wells's great-grandson Simon Wells, with an even more revised plot that incorporated the ideas of paradoxes and changing the past. The place is changed from Richmond, Surrey, to downtown New York City, where the Time Traveller moves 

## Part 2: Building Semantic Search from Scratch

### Get the text archive:

In [14]:
from annoy import AnnoyIndex
import numpy as np
import pandas as pd
import re

In [15]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

### Chunking:

In [16]:
# Split into a list of sentences
texts = text.split(".")

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(" \n") for t in texts])

In [17]:
texts

array(['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
       'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
       'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
       'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
       'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
       'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
       'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',

In [18]:
# Split into a list of paragraphs
texts = text.split("\n\n")

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(" \n") for t in texts])

In [19]:
texts

array(['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.',
       'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstellar uses extensive practical 

In [20]:
# Split into a list of sentences
texts = text.split(".")

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(" \n") for t in texts])

Prepend title to add context

In [21]:
title = "Interstellar (film)"
texts = np.array([f"{title} {t}" for t in texts])

In [22]:
texts

array(['Interstellar (film) Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
       'Interstellar (film) It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
       'Interstellar (film) Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
       'Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
       'Interstellar (film) Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
       'Interstellar (film) Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format

### Get the embeddings:

In [23]:
response = co.embed(
    texts=texts.tolist()
).embeddings

In [24]:
embeds = np.array(response)
embeds.shape

(15, 4096)

### Create the search index:

In [25]:
search_index = AnnoyIndex(f=embeds.shape[1], metric="angular")
# Add all vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i=i, vector=embeds[i])

output_dir = "./output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

search_index.build(n_trees=10)
search_index.save(os.path.join(output_dir, "test.ann"))

True

In [26]:
pd.set_option("display.max_colwidth", None)

def search(query):
    # Get the query's embedding
    query_embed = co.embed(texts=[query]).embeddings

    # Retrieve the nearest neighbours
    similar_item_ids = search_index.get_nns_by_vector(vector=query_embed[0],
                                                      n=3,
                                                      include_distances=True)
    
    # Format the results
    results = pd.DataFrame(data={"texts": texts[similar_item_ids[0]],
                                 "distance": similar_item_ids[1]})
    
    return results


In [27]:
query = "How much did the film make?"
search(query=query)

,texts,distance
0,"Interstellar (film) The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014",1.019055
1,"Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles",1.144951
2,"Interstellar (film) In the United States, it was first released on film stock, expanding to venues using digital projectors",1.167268
